In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
import pandas as pd

# File path (update this to your actual file path)
file_path = r'D:/Desktop/AD/AD_Dataset/Gene_expression/GPL4372.annot'

# Read the file, skipping lines until the platform table begins
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Find the start of the table
for i, line in enumerate(lines):
    if line.strip() == "!platform_table_begin":
        table_start_index = i + 1
        break

# Load the table into a DataFrame
from io import StringIO
data_str = ''.join(lines[table_start_index:])
df = pd.read_csv(StringIO(data_str), sep='\t', dtype=str)
# Extract only 'ID' and 'Gene symbol' columns
# Column names may need trimming based on the exact file format
if 'ID' in df.columns and 'Gene symbol' in df.columns:
    selected_df = df[['ID', 'Gene symbol']]
else:
    # Try alternative column names if not matched
    print("Available columns:", df.columns.tolist())
    raise ValueError("Could not find 'ID' or 'Gene symbol' columns")

# Preview
print(selected_df.head())


            ID Gene symbol
0  10025930146         NaN
1  10025930335         XPA
2  10025913794   LOC439911
3  10023807248     SLC35A3
4  10023809851     KATNAL1


In [3]:
selected_df[13465:13467]

,ID,Gene symbol
13465,10025915123,NaN
13466,10023813203,APOE


In [4]:
gene_name ='APOE'
# Clean data
selected_df['Gene symbol'] = selected_df['Gene symbol'].astype(str).str.strip()

# Then check
if gene_name in selected_df['Gene symbol'].values:
    print(f"{gene_name} exists in the dataset.")
else:
    print(f"{gene_name} not found in the dataset.")

APOE exists in the dataset.


C:\Users\shaha\AppData\Local\Temp\ipykernel_12932\484126483.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Gene symbol'] = selected_df['Gene symbol'].astype(str).str.strip()


In [5]:
# Strip whitespace and search for 'APOE' in Gene symbol column
selected_df['Gene symbol'] = selected_df['Gene symbol'].astype(str).str.strip()
apoe_rows = selected_df[selected_df['Gene symbol'] == 'SERPINA3']
print(apoe_rows)


                ID Gene symbol
8904   10023804954    SERPINA3
25789  10033669027    SERPINA3


C:\Users\shaha\AppData\Local\Temp\ipykernel_12932\2614067779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Gene symbol'] = selected_df['Gene symbol'].astype(str).str.strip()


In [6]:
data=pd.read_csv("D:/Desktop/AD/AD_Dataset/Gene_expression/GSE33000_filtered_file_orginal_AD.csv")

In [7]:
data['disease_status'] = data['disease_status'].apply(lambda x: 1 if x == "Alzheimer's disease" else 0)

In [8]:
data.head()

,Sample,10019475365,10019481149,10019495284,10019687586,10019713746,10019799479,10019809115,10019874890,10019903058,...,10033669040,10033669041,10033669042,10033669043,10033669045,10033669046,10033669047,10033669048,10033669049,disease_status
0,GSM1423780,-0.016184,-0.017105,0.077440,0.033595,-0.001819,-0.065462,-0.042255,-0.000237,-0.051375,...,-0.005318,-0.081350,0.123740,0.104146,0.114913,0.024853,-0.062032,-0.068883,0.170841,1
1,GSM1423781,0.027599,-0.023574,-0.058746,0.032914,0.032487,0.016333,0.003896,0.026999,0.237243,...,0.045702,0.031939,0.027814,-0.010796,-0.219624,-0.001018,-0.042930,-0.072794,0.154764,1
2,GSM1423782,-0.089661,0.044863,-0.120759,-0.030928,-0.015073,-0.008490,0.005585,0.034198,0.291271,...,0.048495,-0.033006,0.089244,-0.031901,-0.205338,0.039534,-0.175094,-0.003799,0.359424,1
3,GSM1423783,-0.020493,0.003438,0.072692,-0.033051,-0.010303,-0.029326,-0.019656,0.023072,-0.023117,...,0.086333,-0.016658,0.011806,-0.027000,-0.065482,0.031740,-0.037869,-0.046252,0.203776,1
4,GSM1423784,-0.051699,-0.064108,-0.051002,-0.049156,0.063070,0.020476,-0.038470,0.035463,-0.152237,...,0.048858,-0.128627,0.041769,-0.036706,-0.085186,0.028517,-0.088991,0.010318,0.153241,1


In [9]:
data.shape

(467, 39282)

In [10]:


# Check for missing values
print(data.isnull().sum())       # Count missing values in each column
print(data.isnull().any())       # True if any missing in each column
print(data.isnull().values.any())  # True if any missing in entire DataFrame


Sample            0
10019475365       0
10019481149       0
10019495284       0
10019687586       0
                 ..
10033669046       0
10033669047       0
10033669048       0
10033669049       0
disease_status    0
Length: 39282, dtype: int64
Sample            False
10019475365       False
10019481149       False
10019495284       False
10019687586       False
                  ...  
10033669046       False
10033669047       False
10033669048       False
10033669049       False
disease_status    False
Length: 39282, dtype: bool
True


In [11]:
# Drop columns with NaN names
data = data.loc[:, ~data.columns.isnull()]


In [12]:
data.shape

(467, 39282)

In [13]:
data.fillna(0, inplace=True)  # Fill all missing values with 0



In [14]:
#ANOVA

In [15]:
data.isnull().values.any()


False

In [16]:
data.isnull().sum()


Sample            0
10019475365       0
10019481149       0
10019495284       0
10019687586       0
                 ..
10033669046       0
10033669047       0
10033669048       0
10033669049       0
disease_status    0
Length: 39282, dtype: int64

In [17]:
A1=data.iloc[:,-1]
A1.shape
A2=data.iloc[:,1:39281]

In [18]:
A2

,10019475365,10019481149,10019495284,10019687586,10019713746,10019799479,10019809115,10019874890,10019903058,10019909307,...,10033669039,10033669040,10033669041,10033669042,10033669043,10033669045,10033669046,10033669047,10033669048,10033669049
0,-0.016184,-0.017105,0.077440,0.033595,-0.001819,-0.065462,-0.042255,-0.000237,-0.051375,0.031249,...,-0.107363,-0.005318,-0.081350,0.123740,0.104146,0.114913,0.024853,-0.062032,-0.068883,0.170841
1,0.027599,-0.023574,-0.058746,0.032914,0.032487,0.016333,0.003896,0.026999,0.237243,0.047474,...,-0.016103,0.045702,0.031939,0.027814,-0.010796,-0.219624,-0.001018,-0.042930,-0.072794,0.154764
2,-0.089661,0.044863,-0.120759,-0.030928,-0.015073,-0.008490,0.005585,0.034198,0.291271,0.025229,...,0.000134,0.048495,-0.033006,0.089244,-0.031901,-0.205338,0.039534,-0.175094,-0.003799,0.359424
3,-0.020493,0.003438,0.072692,-0.033051,-0.010303,-0.029326,-0.019656,0.023072,-0.023117,0.077798,...,-0.028737,0.086333,-0.016658,0.011806,-0.027000,-0.065482,0.031740,-0.037869,-0.046252,0.203776
4,-0.051699,-0.064108,-0.051002,-0.049156,0.063070,0.020476,-0.038470,0.035463,-0.152237,0.083566,...,-0.063595,0.048858,-0.128627,0.041769,-0.036706,-0.085186,0.028517,-0.088991,0.010318,0.153241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0.018263,0.014296,-0.173077,0.005083,0.030127,0.007436,-0.003252,0.005560,0.206365,0.042491,...,-0.067133,0.086486,-0.026522,0.098675,-0.044785,-0.234285,0.095325,-0.092283,-0.032478,0.136840
463,0.121809,0.010552,-0.258932,-0.144517,0.085608,0.086100,0.051226,0.050017,0.009005,-0.036068,...,0.011189,0.127658,-0.007401,0.082875,-0.074809,-0.264415,0.143286,-0.008689,-0.009868,-0.072675
464,-0.034938,-0.010118,-0.159735,-0.068259,-0.009837,0.025641,-0.023850,0.015296,0.062978,0.071460,...,-0.036274,0.048522,-0.105815,0.061433,0.005608,-0.274293,0.071590,-0.180909,-0.062862,0.115837
465,0.073973,0.017207,-0.182124,-0.074872,0.057851,0.015129,0.042788,0.057272,-0.064845,0.006922,...,-0.032576,0.037349,-0.005210,0.115882,-0.070878,-0.274968,0.122321,-0.073528,-0.020370,0.080425


In [19]:
A1

0      1
1      1
2      1
3      1
4      1
      ..
462    0
463    0
464    0
465    0
466    0
Name: disease_status, Length: 467, dtype: int64

In [20]:
selected_columns1 = A2.columns[:39280]
scaler1 = StandardScaler()
scaled_array1 = scaler1.fit_transform(A2)

# Convert back to DataFrame
X_scaled1 = pd.DataFrame(scaled_array1, columns=selected_columns1)

# If you want to replace the original columns in df
X_scaled1[selected_columns1] = X_scaled1

In [21]:
#ANOVA
from sklearn.feature_selection import SelectKBest, f_classif
# Apply ANOVA F-test
selector = SelectKBest(score_func=f_classif, k='all')  # k='all' gets scores for all features
X_new = selector.fit_transform(A2, A1)

# Get scores
scores = selector.scores_
p_values = selector.pvalues_

# Display scores and p-values
anova_results = pd.DataFrame({'Feature': A2.columns, 'F_Score': scores, 'p_Value': p_values})
anova_results = anova_results.sort_values(by='F_Score', ascending=False)

print(anova_results)


           Feature       F_Score       p_Value
4181   10023815322  6.705886e+02  3.354598e-92
36773  10025934933  5.923533e+02  5.555271e-85
7108   10023822632  5.835409e+02  3.901302e-84
15967  10025904757  5.739465e+02  3.318828e-83
37289  10026391091  5.524370e+02  4.336352e-81
...            ...           ...           ...
11651  10023841134  4.096243e-08  9.998386e-01
22768  10025911673  3.506584e-08  9.998507e-01
25902  10025915897  3.005436e-08  9.998618e-01
21625  10025910508  2.653806e-08  9.998701e-01
5141   10023817671  1.784383e-08  9.998935e-01

[39280 rows x 3 columns]


In [22]:
# Sort the dataframe by F_Score in descending order
df_sorted =anova_results.sort_values(by='F_Score', ascending=False)

# Extract top 200 rows
selected_ids= df_sorted.head(20)

# Extract only the ID column (assuming it’s named 'Feature')
selected_ids = selected_ids['Feature'].tolist()

# Show or save
print(selected_ids)


['10023815322', '10025934933', '10023822632', '10025904757', '10026391091', '10025910121', '10025911097', '10023815084', '10025906503', '10023812404', '10023810409', '10025902315', '10023816018', '10025908435', '10025905029', '10025903459', '10025904476', '10033668850', '10025923114', '10025931577']


In [23]:
selected_ids 

['10023815322',
 '10025934933',
 '10023822632',
 '10025904757',
 '10026391091',
 '10025910121',
 '10025911097',
 '10023815084',
 '10025906503',
 '10023812404',
 '10023810409',
 '10025902315',
 '10023816018',
 '10025908435',
 '10025905029',
 '10025903459',
 '10025904476',
 '10033668850',
 '10025923114',
 '10025931577']

In [24]:
# Keep Sample and disease_status in the same position
final_columns = ["Sample", "disease_status"] + selected_ids

# Extract only those columns
df_selected = data[final_columns]

print(df_selected.shape)

(467, 22)


In [25]:
ya=df_selected.iloc[:,1]
xa=df_selected.iloc[:,2:22]

In [26]:
xa.head()

,10023815322,10025934933,10023822632,10025904757,10026391091,10025910121,10025911097,10023815084,10025906503,10023812404,10023810409,10025902315,10023816018,10025908435,10025905029,10025903459,10025904476,10033668850,10025923114,10025931577
0,-0.640795,-0.031223,-0.113261,-0.131614,-0.247186,-0.211608,-0.067676,-0.039479,-0.195782,0.026720,-0.195314,-0.016729,-0.345573,-0.400771,-0.139552,-0.027576,0.130576,0.167544,-0.303719,-0.047784
1,-0.502130,0.007374,-0.146216,-0.074533,0.013891,0.069601,0.091287,-0.020911,-0.022373,-0.032433,-0.184518,-0.106007,-0.157141,-0.135038,-0.024162,-0.000315,-0.082315,-0.024495,-0.139142,-0.002079
2,0.250133,0.111907,0.204772,0.255189,0.141044,0.148990,0.303958,0.189568,-0.469899,-0.149040,0.326574,-0.358417,0.341646,0.382709,-0.233169,-0.098543,-0.476269,-0.387251,0.365051,0.074431
3,-0.161392,0.018001,-0.065107,-0.102461,0.169786,0.007809,-0.093091,-0.091290,0.176772,0.032835,-0.226637,0.081531,0.171019,0.191234,0.114795,-0.008427,0.144072,0.178451,0.282808,-0.306889
4,-0.340519,0.014440,-0.083818,-0.122204,0.006845,0.072550,0.005769,-0.030681,-0.340925,0.008845,-0.046116,-0.173503,-0.079580,-0.047080,-0.271388,-0.008664,-0.209363,-0.127408,-0.050313,-0.064495


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [28]:
# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(xa, ya, test_size=0.2, random_state=42, stratify=ya)


#Random_ Forest wirh ANOVA
rf = RandomForestClassifier(random_state=42)

# Define Grid Search Parameters
param_grid = {
    'n_estimators': [200],
    'max_depth': [6],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [False]
}

# Setup GridSearch with 5-fold CV
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           scoring='accuracy',
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)

# Best model
best_rf = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_rf.predict(X_test)
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'bootstrap': False, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Test Set Accuracy: 0.9148936170212766

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87        32
           1       0.92      0.95      0.94        62

    accuracy                           0.91        94
   macro avg       0.91      0.90      0.90        94
weighted avg       0.91      0.91      0.91        94



In [29]:
#ANOVA AND RF-HYBRID

In [30]:
# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(xa, ya, test_size=0.2, random_state=42, stratify=ya)

rf = RandomForestClassifier(n_estimators=300, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
svc = SVC(probability=True, kernel="rbf", random_state=42)

stack_model = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    final_estimator=LogisticRegression(),
    cv=5
)
stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)

print("Stacking Ensemble Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

voting_model = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    voting='soft'   # soft = probability average
)

voting_model.fit(X_train, y_train)
y_pred_voting = voting_model.predict(X_test)

print("Voting Ensemble Accuracy:", accuracy_score(y_test, y_pred_voting))

C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.

Stacking Ensemble Accuracy: 0.9042553191489362

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.81      0.85        32
           1       0.91      0.95      0.93        62

    accuracy                           0.90        94
   macro avg       0.90      0.88      0.89        94
weighted avg       0.90      0.90      0.90        94



C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Voting Ensemble Accuracy: 0.9148936170212766


In [31]:
#ANOVA AND MLP

In [32]:

# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(xa, ya, test_size=0.2, random_state=42, stratify=ya)

# MLP Classifier
mlp = MLPClassifier(random_state=42, max_iter=1000)

# Grid Search parameters
param_grid = {
    'hidden_layer_sizes': [ (200,100,100,50,50)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant']
}

# GridSearchCV with 5-fold CV
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)
# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)
# Best Model
best_mlp = grid_search.best_estimator_

# Predictions
y_pred = best_mlp.predict(X_test)
y_proba = best_mlp.predict_proba(X_test)[:, 1]

# Accuracy
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))



Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 100, 100, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}

Test Set Accuracy: 0.8936170212765957

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.81      0.84        32
           1       0.91      0.94      0.92        62

    accuracy                           0.89        94
   macro avg       0.89      0.87      0.88        94
weighted avg       0.89      0.89      0.89        94



In [33]:
#RELIE

In [34]:
xr=data.iloc[:,1:39281]
yr=data.iloc[:,-1]

In [35]:
yr

0      1
1      1
2      1
3      1
4      1
      ..
462    0
463    0
464    0
465    0
466    0
Name: disease_status, Length: 467, dtype: int64

In [36]:
scaler= StandardScaler()
scaled = scaler1.fit_transform(xr)

In [37]:
data.shape

(467, 39282)

In [38]:
data1=pd.read_csv("D:/Desktop/AD/AD_Dataset/Gene_expression/DEG_results_AD_vs_Control_569.csv")

In [39]:
data1.head()

,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
0,10023815322,-0.895987,-0.500405,-25.975125,9.160000e-93,3.600000e-88,200.849912
1,10023822632,-0.435269,-0.172190,-24.221509,1.320000e-84,1.730000e-80,182.176631
2,10025904757,-0.318385,-0.144647,-24.011139,1.270000e-83,1.250000e-79,179.924028
3,10026391091,-0.547300,-0.218477,-23.572041,1.450000e-81,1.140000e-77,175.215392
4,10025910121,-0.505063,-0.202270,-23.397297,9.590000e-81,6.280000e-77,173.339192


In [40]:
data1.shape

(569, 7)

In [41]:

# Step 2: Extract DEG IDs
deg_ids = data1['ID'].astype(str).tolist()  # Convert to string if not already

# Step 3: Keep 'Sample' and 'Disease Status' columns
meta_cols = ['Sample', 'disease_status'] if 'disease_status' in data.columns else ['Sample']

# Step 4: Filter columns by matching DEG IDs (intersection)
matched_cols = [col for col in data.columns if col in deg_ids]

# Step 5: Create filtered DataFrame
filtered_data = data[meta_cols + matched_cols]

# Step 6: Check result
print(filtered_data.head())


       Sample  disease_status  10023804739  10023804764  10023804954  \
0  GSM1423780               1     0.177249    -0.088015     0.295011   
1  GSM1423781               1     0.270190    -0.000432    -0.546749   
2  GSM1423782               1    -0.131901    -0.176698    -1.072475   
3  GSM1423783               1     0.101474    -0.081289     0.304003   
4  GSM1423784               1    -0.220523    -0.153983    -0.998865   

   10023804956  10023805010  10023805215  10023805227  10023805680  ...  \
0    -0.314823    -0.169116     0.065891     0.025851    -0.124535  ...   
1    -0.082688    -0.032081     0.047376    -0.008422    -0.061468  ...   
2     0.157847     0.156469     0.155209    -0.187734     0.168977  ...   
3    -0.355276    -0.114416    -0.019706     0.336080    -0.105837  ...   
4    -0.208425    -0.039808     0.110388    -0.430443    -0.038451  ...   

   10033668859  10033668881  10033668900  10033668905  10033668934  \
0    -0.118848     0.378457     0.295191    -0

In [42]:
filtered_data.head()

,Sample,disease_status,10023804739,10023804764,10023804954,10023804956,10023805010,10023805215,10023805227,10023805680,...,10033668859,10033668881,10033668900,10033668905,10033668934,10033668947,10033668969,10033669027,10033669028,10033669034
0,GSM1423780,1,0.177249,-0.088015,0.295011,-0.314823,-0.169116,0.065891,0.025851,-0.124535,...,-0.118848,0.378457,0.295191,-0.195486,-0.187331,0.148425,-0.108735,0.282643,0.222258,0.378551
1,GSM1423781,1,0.270190,-0.000432,-0.546749,-0.082688,-0.032081,0.047376,-0.008422,-0.061468,...,0.021625,-0.008306,0.183570,0.147839,-0.083502,0.114653,0.032929,-0.489616,0.032550,-1.773067
2,GSM1423782,1,-0.131901,-0.176698,-1.072475,0.157847,0.156469,0.155209,-0.187734,0.168977,...,0.123719,-0.434719,-0.129282,-0.429158,-0.564054,-0.247031,-0.012166,-0.942820,-0.370841,-1.871089
3,GSM1423783,1,0.101474,-0.081289,0.304003,-0.355276,-0.114416,-0.019706,0.336080,-0.105837,...,-0.186090,0.154657,0.214771,0.357409,-0.036166,-0.004077,-0.029744,0.307587,0.125360,0.338872
4,GSM1423784,1,-0.220523,-0.153983,-0.998865,-0.208425,-0.039808,0.110388,-0.430443,-0.038451,...,-0.069031,-0.176467,-0.182047,-0.168579,-0.306143,-0.186886,-0.634957,-0.872797,-0.300245,0.432422


In [43]:
filtered_data.shape

(467, 571)

In [44]:
#filtered_data.to_csv("d:/Desktop/AD/AD_Dataset/Gene_expression/gene_45_last_genes.csv", index=False)

In [45]:
#filtered_data.to_csv("d:/Desktop/AD/AD_Dataset/Gene_expression/gene_45_last_genes.csv")

In [46]:
y=filtered_data.iloc[:,1]
y.shape
X1=filtered_data.iloc[:,2:571]

In [47]:
y

0      1
1      1
2      1
3      1
4      1
      ..
462    0
463    0
464    0
465    0
466    0
Name: disease_status, Length: 467, dtype: int64

In [48]:
X1.head()

,10023804739,10023804764,10023804954,10023804956,10023805010,10023805215,10023805227,10023805680,10023805805,10023805852,...,10033668859,10033668881,10033668900,10033668905,10033668934,10033668947,10033668969,10033669027,10033669028,10033669034
0,0.177249,-0.088015,0.295011,-0.314823,-0.169116,0.065891,0.025851,-0.124535,0.074911,0.041636,...,-0.118848,0.378457,0.295191,-0.195486,-0.187331,0.148425,-0.108735,0.282643,0.222258,0.378551
1,0.270190,-0.000432,-0.546749,-0.082688,-0.032081,0.047376,-0.008422,-0.061468,0.295260,0.012569,...,0.021625,-0.008306,0.183570,0.147839,-0.083502,0.114653,0.032929,-0.489616,0.032550,-1.773067
2,-0.131901,-0.176698,-1.072475,0.157847,0.156469,0.155209,-0.187734,0.168977,-0.083465,-0.260322,...,0.123719,-0.434719,-0.129282,-0.429158,-0.564054,-0.247031,-0.012166,-0.942820,-0.370841,-1.871089
3,0.101474,-0.081289,0.304003,-0.355276,-0.114416,-0.019706,0.336080,-0.105837,-0.090805,0.184364,...,-0.186090,0.154657,0.214771,0.357409,-0.036166,-0.004077,-0.029744,0.307587,0.125360,0.338872
4,-0.220523,-0.153983,-0.998865,-0.208425,-0.039808,0.110388,-0.430443,-0.038451,-0.232449,-0.275562,...,-0.069031,-0.176467,-0.182047,-0.168579,-0.306143,-0.186886,-0.634957,-0.872797,-0.300245,0.432422


In [49]:
X1.shape

(467, 569)

In [50]:
import pandas as pd

# Create a dictionary: ID → Gene symbol
id_to_gene = dict(zip(selected_df['ID'].astype(str), selected_df['Gene symbol']))

# Replace column names (only if column name is in the mapping)
new_columns = []
for col in X1.columns:
    if col in id_to_gene:
        new_name = id_to_gene[col]
        new_columns.append(new_name if pd.notna(new_name) and new_name != '' else col)  # keep original if NaN
    else:
        new_columns.append(col)  # Keep original if no match

# Apply new column names
X1.columns = new_columns

# ✅ Preview updated DataFrame
print(X1.head())

# Optional: Save to new file
# expression_df.to_csv('renamed_expression_data.csv', index=False)


     MS4A4A      IL18  SERPINA3     FREM3    CKMT1B       nan   GADD45B  \
0  0.177249 -0.088015  0.295011 -0.314823 -0.169116  0.065891  0.025851   
1  0.270190 -0.000432 -0.546749 -0.082688 -0.032081  0.047376 -0.008422   
2 -0.131901 -0.176698 -1.072475  0.157847  0.156469  0.155209 -0.187734   
3  0.101474 -0.081289  0.304003 -0.355276 -0.114416 -0.019706  0.336080   
4 -0.220523 -0.153983 -0.998865 -0.208425 -0.039808  0.110388 -0.430443   

        HN1      MRC1      SRGN  ...      RBP4       CFI  TNFRSF11B      MT1X  \
0 -0.124535  0.074911  0.041636  ... -0.118848  0.378457   0.295191 -0.195486   
1 -0.061468  0.295260  0.012569  ...  0.021625 -0.008306   0.183570  0.147839   
2  0.168977 -0.083465 -0.260322  ...  0.123719 -0.434719  -0.129282 -0.429158   
3 -0.105837 -0.090805  0.184364  ... -0.186090  0.154657   0.214771  0.357409   
4 -0.038451 -0.232449 -0.275562  ... -0.069031 -0.176467  -0.182047 -0.168579   

   TNFRSF10D       CFH   SLC22A2  SERPINA3    CYP1B1      XIST

In [51]:
X1.head()

,MS4A4A,IL18,SERPINA3,FREM3,CKMT1B,nan,GADD45B,HN1,MRC1,SRGN,...,RBP4,CFI,TNFRSF11B,MT1X,TNFRSF10D,CFH,SLC22A2,SERPINA3,CYP1B1,XIST
0,0.177249,-0.088015,0.295011,-0.314823,-0.169116,0.065891,0.025851,-0.124535,0.074911,0.041636,...,-0.118848,0.378457,0.295191,-0.195486,-0.187331,0.148425,-0.108735,0.282643,0.222258,0.378551
1,0.270190,-0.000432,-0.546749,-0.082688,-0.032081,0.047376,-0.008422,-0.061468,0.295260,0.012569,...,0.021625,-0.008306,0.183570,0.147839,-0.083502,0.114653,0.032929,-0.489616,0.032550,-1.773067
2,-0.131901,-0.176698,-1.072475,0.157847,0.156469,0.155209,-0.187734,0.168977,-0.083465,-0.260322,...,0.123719,-0.434719,-0.129282,-0.429158,-0.564054,-0.247031,-0.012166,-0.942820,-0.370841,-1.871089
3,0.101474,-0.081289,0.304003,-0.355276,-0.114416,-0.019706,0.336080,-0.105837,-0.090805,0.184364,...,-0.186090,0.154657,0.214771,0.357409,-0.036166,-0.004077,-0.029744,0.307587,0.125360,0.338872
4,-0.220523,-0.153983,-0.998865,-0.208425,-0.039808,0.110388,-0.430443,-0.038451,-0.232449,-0.275562,...,-0.069031,-0.176467,-0.182047,-0.168579,-0.306143,-0.186886,-0.634957,-0.872797,-0.300245,0.432422


In [52]:
print(X1.columns.tolist())


['MS4A4A', 'IL18', 'SERPINA3', 'FREM3', 'CKMT1B', 'nan', 'GADD45B', 'HN1', 'MRC1', 'SRGN', 'TNFRSF11B', 'EPB41L3', 'PTGDS', 'EGR2', 'EGR3', 'CFHR1', 'CFB', 'TMEFF1', 'MAP1B', 'SPP1', 'GBP2', 'HSPA1A', 'BAG3', 'SCG3', 'ATP6V1A', 'nan', 'F13A1', 'LRRC32', 'LMO3', 'CBLN4', 'LAT2', 'KCNJ3', 'TAC1', 'DDX3Y', 'SLN', 'TMSB4Y', 'CD44', 'AEBP1', 'EGR1', 'NETO2', 'PAK3', 'ADM', 'DCLK1', 'CRYM', 'GFAP', 'KCNK1', 'MT1G', 'D4S234E', 'ACTN2', 'S100A9', 'VIM', 'NEK2', 'MGP', 'GNG2', 'DEFB1', 'LCP1', 'TXNIP', 'nan', 'nan', 'RNASE2', 'NMU', 'CA10', 'IFITM2', 'MID1IP1', 'MAP2K1', 'VIP', 'nan', 'FBXO9', 'CRHBP', 'MFSD1', 'SLC16A9', 'TIMP1', 'GYPC', 'TAC1', 'S100A11', 'GRP', 'SKIP', 'HPRT1', 'nan', 'CIRBP', 'NFE2', 'ZIC1', 'ADCYAP1', 'CDH8', 'PCP4L1', 'PYGL', 'PCDH8', 'RGS4', 'TBX15', 'SST', 'S100A10', 'CAP2', 'FGL2', 'CYP26A1', 'HSPB1', 'TAC1', 'RAB13', 'nan', 'ATP6V1B2', 'nan', 'NUPR1', 'ATF7IP2', 'ADD2', 'PVALB', 'C1S', 'UCHL1', 'SSTR1', 'SPON2', 'TBC1D9', 'CRABP1', 'ITGA5', 'nan', 'CD14', 'BEX1', 'HSP

In [53]:
X1.shape

(467, 569)

In [54]:
# Check for missing values per column
missing_values = X1.isnull().sum()

# Filter only the columns that have missing values
missing_columns = missing_values[missing_values > 0]

# Display missing columns and their missing value count
print("Missing values per column:")
print(missing_columns)


Missing values per column:
Series([], dtype: int64)


In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [56]:
# Find duplicate column names
duplicates = X1.columns[X1.columns.duplicated()]
print("Duplicate columns:", duplicates)

# Remove duplicated columns (keep the first occurrence)
X1 = X1.loc[:, ~X1.columns.duplicated()]


Duplicate columns: Index(['nan', 'nan', 'nan', 'nan', 'TAC1', 'nan', 'TAC1', 'nan', 'nan', 'nan',
       ...
       'C4A', 'RBP4', 'CFI', 'TNFRSF11B', 'MT1X', 'TNFRSF10D', 'CFH',
       'SERPINA3', 'CYP1B1', 'XIST'],
      dtype='object', length=118)


In [57]:
X1.head()

,MS4A4A,IL18,SERPINA3,FREM3,CKMT1B,nan,GADD45B,HN1,MRC1,SRGN,...,FCGR1B,USP9Y,FLJ45910,ANXA2P1,TNFRSF1B,SLC7A7,TNFRSF1A,TNFRSF6B,SLC26A2,SLC22A2
0,0.177249,-0.088015,0.295011,-0.314823,-0.169116,0.065891,0.025851,-0.124535,0.074911,0.041636,...,0.178558,-1.537239,-0.177974,0.030255,0.013054,-0.088667,0.028669,-0.013683,-0.001535,-0.108735
1,0.270190,-0.000432,-0.546749,-0.082688,-0.032081,0.047376,-0.008422,-0.061468,0.295260,0.012569,...,0.153275,0.276355,0.369547,-0.122778,0.025586,-0.109704,-0.023243,0.261750,0.048704,0.032929
2,-0.131901,-0.176698,-1.072475,0.157847,0.156469,0.155209,-0.187734,0.168977,-0.083465,-0.260322,...,-0.292065,0.320073,-0.186797,-0.194351,-0.278598,-0.201195,-0.335403,-0.416407,-0.156293,-0.012166
3,0.101474,-0.081289,0.304003,-0.355276,-0.114416,-0.019706,0.336080,-0.105837,-0.090805,0.184364,...,0.265761,-1.567426,0.163525,0.165267,0.102365,-0.028988,0.246976,0.574352,-0.005429,-0.029744
4,-0.220523,-0.153983,-0.998865,-0.208425,-0.039808,0.110388,-0.430443,-0.038451,-0.232449,-0.275562,...,0.051617,-1.834807,0.198740,-0.341765,-0.158555,-0.137937,-0.186750,-0.195191,-0.099679,-0.634957


In [58]:
X1.to_csv("d:/Desktop/AD/AD_Dataset/Gene_expression/gene_451_last1_genes.csv", index=False)

In [59]:
y

0      1
1      1
2      1
3      1
4      1
      ..
462    0
463    0
464    0
465    0
466    0
Name: disease_status, Length: 467, dtype: int64

In [60]:

scaler = StandardScaler()
X1_scaled_array = scaler.fit_transform(X1)

# Create scaled DataFrame with same column names and index
X1_scaled = pd.DataFrame(X1_scaled_array, columns=X1.columns, index=X1.index)

# Check the result
print(X1_scaled.head())


     MS4A4A      IL18  SERPINA3     FREM3    CKMT1B       nan   GADD45B  \
0  0.646429 -0.159480  1.274295 -0.200141  0.093351  0.786348  0.121576   
1  0.939348  0.109625 -0.161783  0.480957  0.556347  0.720744  0.011099   
2 -0.327907 -0.431961 -1.058691  1.186707  1.193392  1.102816 -0.566910   
3  0.407613 -0.138812  1.289634 -0.318835  0.278163  0.483060  1.121593   
4 -0.607213 -0.362169 -0.933110  0.112038  0.530241  0.944006 -1.349274   

        HN1      MRC1      SRGN  ...    FCGR1B     USP9Y  FLJ45910   ANXA2P1  \
0  0.327801  0.266192  0.123472  ...  1.022433 -1.476495 -1.188472  0.050283   
1  0.594600  0.882997  0.030669  ...  0.946350  0.882282  0.189857 -0.424444   
2  1.569480 -0.177139 -0.840593  ... -0.393797  0.939142 -1.210685 -0.646473   
3  0.406900 -0.197683  0.579165  ...  1.284850 -1.515757 -0.328783  0.469106   
4  0.691972 -0.594175 -0.889253  ...  0.640433 -1.863515 -0.240135 -1.103773   

   TNFRSF1B    SLC7A7  TNFRSF1A  TNFRSF6B   SLC26A2   SLC22A2  
0 -0

In [61]:
print(len(X1))

467


In [62]:
#DEG-RF

In [63]:
# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(X1_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [64]:
#Random_ Forest
rf = RandomForestClassifier(random_state=42)

# Define Grid Search Parameters
param_grid = {
    'n_estimators': [200],
    'max_depth': [6],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [False]
}

# Setup GridSearch with 5-fold CV
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,
                           n_jobs=-1,
                           scoring='accuracy',
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)

# Best model
best_rf = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_rf.predict(X_test)
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'bootstrap': False, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Test Set Accuracy: 0.8829787234042553

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.81      0.83        32
           1       0.90      0.92      0.91        62

    accuracy                           0.88        94
   macro avg       0.87      0.87      0.87        94
weighted avg       0.88      0.88      0.88        94



In [78]:
X1.shape

(467, 451)

In [81]:
X_test.shape

(94, 451)

In [ ]:
#SHAP ANALYSIS

In [80]:
import shap

# =====================
X_train, X_test, y_train, y_test = train_test_split(
    X1, y, test_size=0.2, random_state=42, stratify=y
)

# =====================
# 3. Train Random Forest
# =====================
rf = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# =====================
# 4. SHAP Analysis
# =====================
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

# Check shape
print("shap_values[0] shape:", shap_values[0].shape)
print("shap_values[1] shape:", shap_values[1].shape)
print("X_test shape:", X_test.shape)

# =====================
# 5. Display Plots
# =====================
# Use shap_values[1] if you're interested in the "disease" class
shap.summary_plot(shap_values[1], X_test, plot_type="bar")
shap.summary_plot(shap_values[1], X_test)

# Example: Force plot for first test sample
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][0,:],
    X_test.iloc[0,:],
    matplotlib=True
)


shap_values[0] shape: (451, 2)
shap_values[1] shape: (451, 2)
X_test shape: (94, 451)


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

In [65]:
#DEG-RF-HYBRID

In [66]:
X_train, X_test, y_train, y_test = train_test_split(
   X1_scaled, y, test_size=0.2, random_state=42, stratify=y
)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
svc = SVC(probability=True, kernel="rbf", random_state=42)

# -------------------------------
# Stacking Ensemble
# -------------------------------
stack_model = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    final_estimator=LogisticRegression(),
    cv=5
)

# -------------------------------
# Fit and evaluate
# -------------------------------
stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)
print("Stacking Ensemble Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
# -------------------------------
# Voting Classifier for comparison
# -------------------------------
voting_model = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    voting='soft'   # soft = probability average
)

voting_model.fit(X_train, y_train)
y_pred_voting = voting_model.predict(X_test)

print("Voting Ensemble Accuracy:", accuracy_score(y_test, y_pred_voting))


C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:12:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.

Stacking Ensemble Accuracy: 0.9042553191489362

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.84      0.86        32
           1       0.92      0.94      0.93        62

    accuracy                           0.90        94
   macro avg       0.90      0.89      0.89        94
weighted avg       0.90      0.90      0.90        94



C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:13:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Voting Ensemble Accuracy: 0.9042553191489362


In [67]:
#DEG-MLP

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
   X1_scaled, y, test_size=0.2, random_state=42, stratify=y
)


# MLP Classifier
mlp = MLPClassifier(random_state=42, max_iter=1000)

# Grid Search parameters
param_grid = {
    'hidden_layer_sizes': [ (200,100,100,50,50)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant']
}

# GridSearchCV with 5-fold CV
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)
# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)
# Best Model
best_mlp = grid_search.best_estimator_

# Predictions
y_pred = best_mlp.predict(X_test)
y_proba = best_mlp.predict_proba(X_test)[:, 1]

# Accuracy
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))



Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 100, 100, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}

Test Set Accuracy: 0.9361702127659575

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        32
           1       0.95      0.95      0.95        62

    accuracy                           0.94        94
   macro avg       0.93      0.93      0.93        94
weighted avg       0.94      0.94      0.94        94



In [69]:
#PCA

In [70]:
X=A2

In [71]:
X

,10019475365,10019481149,10019495284,10019687586,10019713746,10019799479,10019809115,10019874890,10019903058,10019909307,...,10033669039,10033669040,10033669041,10033669042,10033669043,10033669045,10033669046,10033669047,10033669048,10033669049
0,-0.016184,-0.017105,0.077440,0.033595,-0.001819,-0.065462,-0.042255,-0.000237,-0.051375,0.031249,...,-0.107363,-0.005318,-0.081350,0.123740,0.104146,0.114913,0.024853,-0.062032,-0.068883,0.170841
1,0.027599,-0.023574,-0.058746,0.032914,0.032487,0.016333,0.003896,0.026999,0.237243,0.047474,...,-0.016103,0.045702,0.031939,0.027814,-0.010796,-0.219624,-0.001018,-0.042930,-0.072794,0.154764
2,-0.089661,0.044863,-0.120759,-0.030928,-0.015073,-0.008490,0.005585,0.034198,0.291271,0.025229,...,0.000134,0.048495,-0.033006,0.089244,-0.031901,-0.205338,0.039534,-0.175094,-0.003799,0.359424
3,-0.020493,0.003438,0.072692,-0.033051,-0.010303,-0.029326,-0.019656,0.023072,-0.023117,0.077798,...,-0.028737,0.086333,-0.016658,0.011806,-0.027000,-0.065482,0.031740,-0.037869,-0.046252,0.203776
4,-0.051699,-0.064108,-0.051002,-0.049156,0.063070,0.020476,-0.038470,0.035463,-0.152237,0.083566,...,-0.063595,0.048858,-0.128627,0.041769,-0.036706,-0.085186,0.028517,-0.088991,0.010318,0.153241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0.018263,0.014296,-0.173077,0.005083,0.030127,0.007436,-0.003252,0.005560,0.206365,0.042491,...,-0.067133,0.086486,-0.026522,0.098675,-0.044785,-0.234285,0.095325,-0.092283,-0.032478,0.136840
463,0.121809,0.010552,-0.258932,-0.144517,0.085608,0.086100,0.051226,0.050017,0.009005,-0.036068,...,0.011189,0.127658,-0.007401,0.082875,-0.074809,-0.264415,0.143286,-0.008689,-0.009868,-0.072675
464,-0.034938,-0.010118,-0.159735,-0.068259,-0.009837,0.025641,-0.023850,0.015296,0.062978,0.071460,...,-0.036274,0.048522,-0.105815,0.061433,0.005608,-0.274293,0.071590,-0.180909,-0.062862,0.115837
465,0.073973,0.017207,-0.182124,-0.074872,0.057851,0.015129,0.042788,0.057272,-0.064845,0.006922,...,-0.032576,0.037349,-0.005210,0.115882,-0.070878,-0.274968,0.122321,-0.073528,-0.020370,0.080425


In [72]:

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
X = A2   # features (39,000 genes)
y = A1                  # target (AD vs Control)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# keep top 100 components (you can tune this number)
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {X.shape}, After PCA: {X_pca.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

clf = RandomForestClassifier(n_estimators=300, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

explained_var = np.cumsum(pca.explained_variance_ratio_)
print("Variance explained by first 100 PCs:", explained_var[-1])


Original shape: (467, 39280), After PCA: (467, 100)
Accuracy: 0.8829787234042553
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.69      0.80        32
           1       0.86      0.98      0.92        62

    accuracy                           0.88        94
   macro avg       0.91      0.84      0.86        94
weighted avg       0.89      0.88      0.88        94

Variance explained by first 100 PCs: 0.8407887679270308


In [73]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
svc = SVC(probability=True, kernel="rbf", random_state=42)

# -------------------------------
# Stacking Ensemble
# -------------------------------
stack_model = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    final_estimator=LogisticRegression(),
    cv=5
)

# -------------------------------
# Fit and evaluate
# -------------------------------
stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)

print("Stacking Ensemble Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
# -------------------------------
# Voting Classifier for comparison
# -------------------------------
voting_model = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svc', svc)],
    voting='soft'   # soft = probability average
)

voting_model.fit(X_train, y_train)
y_pred_voting = voting_model.predict(X_test)

print("Voting Ensemble Accuracy:", accuracy_score(y_test, y_pred_voting))


C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:13:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:13:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:13:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Stacking Ensemble Accuracy: 0.8723404255319149

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.78      0.81        32
           1       0.89      0.92      0.90        62

    accuracy                           0.87        94
   macro avg       0.86      0.85      0.86        94
weighted avg       0.87      0.87      0.87        94



C:\Users\shaha\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:13:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Voting Ensemble Accuracy: 0.8936170212765957


In [75]:
#MLP  and PCA

In [76]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier

In [77]:

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# MLP Classifier
mlp = MLPClassifier(random_state=42, max_iter=1000)

# Grid Search parameters
param_grid = {
    'hidden_layer_sizes': [ (200,100,100,50,50)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant']
}

# GridSearchCV with 5-fold CV
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)
# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)


# Best Model
best_mlp = grid_search.best_estimator_

# Predictions
y_pred = best_mlp.predict(X_test)
y_proba = best_mlp.predict_proba(X_test)[:, 1]

# Accuracy
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 100, 100, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}

Test Set Accuracy: 0.8829787234042553

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.84      0.83        32
           1       0.92      0.90      0.91        62

    accuracy                           0.88        94
   macro avg       0.87      0.87      0.87        94
weighted avg       0.88      0.88      0.88        94

